## Run App

- This notebook runs and displays the dash app 

### Installing and Importing libraries

In [1]:
# # Installing the required libraries
# !python -m pip install -r info.txt
# !pip install --upgrade dash dash-core-components dash-html-components dash-renderer
# !pip install dash_bootstrap_components
# !pip install ipynb

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import dash
import dash_bootstrap_components as dbc
from dash import callback_context
from dash import Output, Input
from dash import dcc
from dash import html
from dash import dash_table
import plotly.express as px
import plotly.graph_objs as go
import re

In [3]:
# Importing the homePageHeaderTitle, Dataoverview, DataVisualization, data preprocessing
from homepage_header_title import header, title_menu, home_page
from data_overview import data_overview_page
from data_visualization import draw_plots, draw_scatter_plot, draw_violin_plot
from data_pre_processing import pre_process_metadata

### Reading Data Files

In [4]:
# read the data files - chronos.csv, cn.csv, expression.csv, metadata.csv
chronos = pd.read_csv("chronos.csv")
cn = pd.read_csv("cn.csv")
expression = pd.read_csv("expression.csv")
metadata = pd.read_csv("metadata.csv")
metadataFiltered = pre_process_metadata(metadata)

In [5]:
dataset = {"chronos" : chronos, "expression" : expression, "cn" : cn}

### App Layout

In [6]:
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app.config['suppress_callback_exceptions'] = True
colors = {
    'background' : '#FFFFFF',
    'text' : '#7FDFF' 
}

dataframe = {'chronos' : chronos.head(20),
             'cn' : cn.head(20), 
             'expression' : expression.head(20), 
             'metadata' : metadata.head(20)}
dataset = {"chronos" : chronos, "expression" : expression, "cn" : cn}

In [7]:
app.layout = html.Div(style = {
                               'width' : '1365px',
                               },
                      children = [
                          header(),
                          html.Div(style = {"display" : "flex"},
                              children = [
                                  title_menu(),
                                  html.Div(style = {"flex" : "83%"},
                                              id = "page-layout"
                                          )
                              ]
                          )
                      ])

In [8]:
@app.callback(
    Output("layout-plot", "children"),
    Output("scatter-plot-layout-button", "style"),
    Output("violin-plot-layout-button", "style"),
    Input("scatter-plot-layout-button", "n_clicks"),
    Input("violin-plot-layout-button", "n_clicks")
)
def change_plot_layout(scatter_plot_layout_button_n_clicks, violin_plot_layout_button_clicks):
    dark_button_style = {'background-color': '#6E8898FF',
                        'color': '#D3D0CBFF',
                        'height': '40px',
                        'width': '175px',
                        "font-weight" : "bolder",
                        'text-align' : 'left',
                        "font-weight" : "bolder",
                        'padding-left' : '50px',
                        "border" : "none",
                        "border-radius" : "2px"}
    light_button_style = {'background-color': '#FFFFFF',
                        'color': '#6E8898FF',
                        'height': '40px',
                        'width': '175px',
                        "font-weight" : "bolder",
                        'text-align' : 'left',
                        "font-weight" : "bolder",
                        'padding-left' : '50px',
                        "border" : "none",
                        "border-radius" : "2px"}
    changed_id_plot = [p['prop_id'] for p in callback_context.triggered][0]
    if 'scatter-plot-layout-button' in changed_id_plot:
        return draw_scatter_plot(), dark_button_style, light_button_style
    elif 'violin-plot-layout-button' in changed_id_plot:
        return draw_violin_plot(), light_button_style, dark_button_style
    else:
        return draw_scatter_plot(), dark_button_style, light_button_style

In [9]:
@app.callback(
    [
    Output("page-layout", "children"), 
    Output("layout-page-title", "children")
    ],
    [
     Input("dashboard-button", "n_clicks"),
     Input("data-overview-button", "n_clicks"),
     Input("data-visualization-button", "n_clicks")
    ]
)
def display_page(n_click_dashboard, n_click_dataoverview, n_click_datavisulaization):
    changed_id = [p['prop_id'] for p in callback_context.triggered][0]
    if 'dashboard-button' in changed_id:
        return home_page(), html.Div(style = {"color" : "#6E8898FF",
                                 "font-weight" : "bold",
                                 "font-size" : "110%"
                                },
                children = [
                    html.H4("Dashboard Information")
                ]
            )
    elif 'data-overview-button' in changed_id:
        return data_overview_page(), html.Div(style = {"color" : "#6E8898FF",
                                 "font-weight" : "bold",
                                 "font-size" : "110%"},
                children = [
                    html.H4("Data Overview")
                ]
            )
    elif 'data-visualization-button' in changed_id:
        return draw_plots(), html.Div(style = {"color" : "#6E8898FF",
                                 "font-weight" : "bold",
                                 "font-size" : "110%"},
                children = [
                    html.H4("Data Visualization")
                ]
            )
    else:
        return home_page(), html.Div(
                                style = {"color" : "#6E8898FF",
                                 "font-weight" : "bold",
                                 "font-size" : "110%"},
                children = [
                    html.H4("Dashboard Information")
                ]
            )

In [10]:
@app.callback(
    Output("table", "children"), [Input("change-table", "value")]
)
def change_table(value):
    return html.Div(
        children = [
        dash_table.DataTable(
        columns=[{"name": i, "id": i, "selectable": True} for i in dataframe[value].columns],
        data=dataframe[value].to_dict('records'),
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        selected_columns=[],
        selected_rows=[],
        style_cell_conditional = [
            {
                'if' : {'column_id' : c},
                
            } for c in dataframe[value].columns
        ],
        style_data = {
            'color' : 'black',
            'backgroundColor' : 'white',
            'textAlign' : 'left',
        },
        style_data_conditional = [
            {
                'if' : {'row_index' : 'odd'},
                'backgroundColor' : 'rgb(220, 220, 220)',
            }
        ],
        style_header = {
            'backgroundColor' : '#6E8898FF',
            'color' : 'white',
            'fontWeight' : 'bold',
            'textAlign' : 'right'
        }
        )])   

In [11]:
@app.callback(
    Output("violinPlot", "figure"), 
    Input("change-gene","value"),
    Input("change-dataset","value"),
    Input("change-metadata-field-violin","value")
)
def update_violinPlot(change_gene_value, change_dataset_value, change_metadata_field_value):
    df = dataset[change_dataset_value]
    
    fig1 = px.violin(df, y=metadataFiltered[change_metadata_field_value], x=change_gene_value, 
                    color=metadataFiltered[change_metadata_field_value], 
                    box=True, points="all",
                   title="Violin Plot of " + f"{change_gene_value}" + 'Vs' + f'{change_dataset_value}'
                    )
    
    return fig1

In [12]:
@app.callback(
    Output("scatterPlot", "figure"), 
    Input("change-chronos-field","value"),
    Input("change-expression-field","value"),
    Input("change-metadata-field-scatter","value")
)
def update_scatterPlot(change_chronos_field_value, change_expression_field_value, change_metadata_field_value):
    fig2 = px.scatter(chronos, 
                     x = chronos[change_chronos_field_value],
                     y = expression[change_expression_field_value],
                     color = metadataFiltered[change_metadata_field_value],
                     title = "Scatter Plot of " + f"{change_chronos_field_value}" + 'Vs' + f'{change_expression_field_value}')
    return fig2

In [13]:
@app.callback(
    Output("summary-statistics", "children"), 
    Input("change-table", "value")
)
def summary_statistics(change_table_value):
    return html.Div(style = {"display" : "flex"},
         children = [
             html.Div(
                 style = {
                      'backgroundColor' : '#00000',
                      'flex' : '15%',
                      'height' : '155px',
                      'textAlign' : 'center',
                      'box-shadow' : '0 4px 8px 0 rgba(0, 0, 0, 0.4)',
                      'padding' : '2%',
                      'box-sizing' : 'border-box',
                      'border-radius' : '2px'
                  },
                  children = [
                      # Dropdown to select the dataset to display
                      html.Div(style = {'color' : '#6E8898FF',
                                        'font-weight' : 'bold',
                                        'text-align' : 'center',
                                       },
                               children = [
                                   html.H3('Dataset'),
                                   html.H5('{}'.format(change_table_value))
                               ])
                  ]
             ),
             html.Div(style = {"width" : "20px"}),
             html.Div(
                 style = {
                      'backgroundColor' : '#00000',
                      'flex' : '15%',
                      'height' : '155px',
                      'textAlign' : 'center',
                      'box-shadow' : '0 4px 8px 0 rgba(0, 0, 0, 0.4)',
                      'padding' : '1%',
                      'box-sizing' : 'border-box',
                      'border-radius' : '2px'
                  },
                  children = [
                      # Dropdown to select the dataset to display
                      html.Div(style = {'color' : '#6E8898FF',
                                        'font-weight' : 'bold',
                                        'text-align' : 'center'
                                       },
                               children = [
                                   html.H4('Number of Rows'),
                                   html.H6('{}'.format(dataframe[change_table_value].shape[0])),
                                   html.H4('Number of columns'),
                                   html.H6('{}'.format(format(dataframe[change_table_value].shape[1])))
                               ])
                  ]
             ),
             html.Div(style = {"width" : "20px"}),
             html.Div(
                 style = {
                      'backgroundColor' : '#00000',
                      'flex' : '15%',
                      'height' : '155px',
                      'textAlign' : 'center',
                      'box-shadow' : '0 4px 8px 0 rgba(0, 0, 0, 0.4)',
                      'padding' : '1%',
                      'box-sizing' : 'border-box',
                      'border-radius' : '2px'
                  },
                  children = [
                      # Dropdown to select the dataset to display
                      html.Div(style = {'color' : '#6E8898FF',
                                        'font-weight' : 'bold',
                                        'text-align' : 'center'
                                       },
                               children = [
                                   html.H5('Number of Sample IDs'),
                                   html.H6('{}'.format(len(dataframe[change_table_value]["Sample_ID"]))),
                                   html.H5('Number of Genes'),
                                   html.H6('{}'.format(len(dataframe[change_table_value].columns[1:])))
                               ])
                  ]
             ),
         ] 
    )

In [14]:
@app.callback(
    Output("download-dataframe-csv", "data"),
    [Input("btn_csv", "n_clicks"),
    Input("change-table", "value")],
    prevent_initial_call=True,
)
def download_data(n_clicks, change_table_value):
    return dcc.send_data_frame(dataframe[change_table_value].to_csv, f'{change_table_value}' + ".csv")
# draw_scatter_plot(),
# draw_violin_plot()

### Running App

In [ ]:
if __name__ == '__main__':
    app.run_server(debug = True, use_reloader = False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
